In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

In [2]:
# Read a CSV file into a DataFrame
csv_file_path = '../Dataset/XRP_dataset.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file_path)
xrp_df1 = df[:500]

In [3]:
xrp_df1.head()

,open_time,open,high,low,close,volume,close_time
0,1567296300000,0.25808,0.25821,0.25778,0.25787,66709.6,1567296599999
1,1567296600000,0.25787,0.25789,0.25744,0.25751,76896.3,1567296899999
2,1567296900000,0.25751,0.25793,0.25745,0.25774,71280.3,1567297199999
3,1567297200000,0.25778,0.25788,0.25757,0.25781,45598.6,1567297499999
4,1567297500000,0.25781,0.25804,0.25753,0.25804,114789.9,1567297799999


In [4]:
def checkCrossOver(macd, signal_line):

    if macd > signal_line: 
        return 1
    elif macd < signal_line: 
        return -1
    else: 
        return 0

In [5]:
def getATR(df, period):
    
    # Calculate the True Range (TR)
    df['high_low'] = df['high'] - df['low']
    df['high_close'] = abs(df['high'] - df['close'].shift())
    df['low_close'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['high_low', 'high_close', 'low_close']].max(axis=1)

    # Calculate the Average True Range (ATR)
    df['atr'] = df['tr'].rolling(window=period).mean()

    return df

In [6]:
def getADX(df, period):
    high = df['high']
    low  = df['low']

    plus_dm = high.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm = low.diff()
    minus_dm[minus_dm > 0] = 0

    atr = getATR(df, period)

    plus_di = 100 * (plus_dm.ewm(alpha = 1/period).mean()/atr['atr'])
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/period).mean() / atr['atr']))
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (period - 1)) + dx) / period
    adx = adx.ewm(alpha=1/period).mean()
    return plus_di, minus_di, adx


In [7]:
xrp_df1['EMA12'] = xrp_df1['close'].ewm(span=12, adjust=False).mean()
xrp_df1['EMA26'] = xrp_df1['close'].ewm(span=26, adjust=False).mean()
xrp_df1['MACD'] = xrp_df1['EMA12'] - xrp_df1['EMA26']
xrp_df1['Signal_Line'] = xrp_df1['MACD'].ewm(span=9, adjust=False).mean()

xrp_df1['plus_di'] = pd.DataFrame(getADX(df, 6)[0]).rename(columns = {0:'plus_di'})
xrp_df1['minus_di'] = pd.DataFrame(getADX(df, 6)[1]).rename(columns = {0:'minus_di'})
xrp_df1['adx'] = pd.DataFrame(getADX(df, 6)[2]).rename(columns = {0:'adx'})

In [8]:
crossover = []
intersect_list = []
old_intersection = None
for index, row in xrp_df1.iterrows():    

    # Add any additional logic here if needed
    intersection = checkCrossOver(row['MACD'], row['Signal_Line'])
    if old_intersection != intersection:
        intersect_list.append(1)
    else: intersect_list.append(0)

    old_intersection = intersection 


    crossover.append(intersection)

xrp_df1['crossover'] = crossover
xrp_df1['intersection'] = intersect_list

In [9]:
def checkMACD(EMA12, EMA26, MACD, signal_line):
    if EMA12 > EMA26 and MACD < signal_line:
        return 1  # Buy signal
    elif EMA12 < EMA26 and MACD > signal_line:
        return -1  # Sell signal
    else:
        return 0  # No signal
    

In [10]:
def checkADX(plus_di, minus_di, adx):
    if adx > 25 and plus_di > minus_di:
        return 1  # Buy signal
    elif adx > 25 and minus_di > plus_di:
        return -1  # Sell signal
    else:
        return 0  # No signal

In [11]:
def checkSignal(MACD, ADX):
    if MACD * ADX == 1:
        return 1
    else: return 0

In [12]:
def strategy(df):
    df['MACD_Signal'] = df.apply(lambda row: checkMACD(row['EMA12'], row['EMA26'], row['MACD'], row['Signal_Line']), axis=1)
    df['ADX_Signal'] = df.apply(lambda row: checkADX(row['plus_di'], row['minus_di'], row['adx']), axis=1)

    # Combine the signals
    df['Combined_Signal'] = df.apply(lambda row: checkSignal(row['MACD_Signal'], row['ADX_Signal']), axis=1)
    return df

In [16]:
#finalize dataset
xrp_df1 = strategy(xrp_df1)

xrp_df1['returns'] = pd.DataFrame(np.diff(xrp_df1['close'], prepend=[0])).rename(columns = {0:'returns'})

strategy_ret = []
for i in range(len(xrp_df1)):
    strategy_ret.append(xrp_df1['returns'][i] * xrp_df1['Combined_Signal'][i])

xrp_df1['strategy_returns'] = strategy_ret


In [20]:
investment_value = 100
investment_ret = []

for i in range(len(xrp_df1['strategy_returns'])):
    number_of_stocks = math.floor(investment_value/xrp_df1['close'][i])
    returns = number_of_stocks * xrp_df1['strategy_returns'][i]
    investment_ret.append(returns)


investment_ret_df = pd.DataFrame(investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(investment_ret_df['investment_returns']), 2)
profit_percentage = math.floor((total_investment_ret/investment_value)*100)
print('Profit gained from the ADX strategy by investing P1000 in XRP : ' + str(total_investment_ret))
print('Profit percentage of the ADX strategy : ' + str(profit_percentage))

Profit gained from the ADX strategy by investing P1000 in XRP : 0.32
Profit percentage of the ADX strategy : 0
